In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from catboost import Pool
from catboost import CatBoostClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import gensim
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb

In [2]:
train = pd.read_csv("/kaggle/input/ykc-2nd/train.csv")
test = pd.read_csv("/kaggle/input/ykc-2nd/test.csv")
sub = pd.read_csv("/kaggle/input/ykc-2nd/sample_submission.csv")

In [3]:
df = pd.concat([train, test])
df = df.reset_index(drop=True)

In [4]:
df["product_name"] = df["product_name"].apply(lambda words : words.lower().replace(",", "").replace("&", "").split(" "))

In [5]:
## 訓練済みの単語ベクトルを読み込んで，product_nameに含まれる単語をベクトルに変換して平均を取ることで，各product_idに対して特徴量ベクトルを作成する

## gensimで.vecから読み込むときに時間がかかるので，他のnotebookでpickleで保存したものを使用している
model = pd.read_pickle("/kaggle/input/yjfasttext/fasttext_gensim_model.pkl") 

## gensimでvecから読み込む場合（５分ぐらいかかる）
# model = gensim.models.KeyedVectors.load_word2vec_format('../input/ykc-2nd/wiki-news-300d-1M.vec/wiki-news-300d-1M.vec')

from collections import defaultdict
unused_words = defaultdict(int)
def to_vec(x, model):
    v = np.zeros(model.vector_size)
    for w in x:
        try:
            v += model[w] ## 単語が訓練済みモデルのvocabにあったら
        except:
            unused_words[w] += 1 ## ベクトルが存在しなかった単語をメモ
    v = v / (np.sqrt(np.sum(v ** 2)) + 1e-16) ## 長さを1に正規化
    return v    
vecs = df["product_name"].apply(lambda x : to_vec(x, model))
vecs = np.vstack(vecs)
fasttext_pretrain_cols = [f"fasttext_pretrain_vec{k}" for k in range(vecs.shape[1])]
vec_df = pd.DataFrame(vecs, columns=fasttext_pretrain_cols)
df = pd.concat([df, vec_df], axis = 1)

In [6]:
target = "department_id" 
n_split = 5
features = fasttext_pretrain_cols + ["order_rate", "order_dow_mode", "order_hour_of_day_mode"]

In [7]:
train = df[~df[target].isna()]
test = df[df[target].isna()]

In [8]:
X = train.drop([target, 'product_name'], axis=1)
y = train[target]

In [9]:
def objective(trial):
    train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.25)
    param = {
        "objective": "multiclass",
        "num_class":21,
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    model = lgb.LGBMClassifier(**param)
    model.fit(train_x, train_y, )
    pred_val = model.predict_proba(valid_x)
    ## evaluate
    score = f1_score(valid_y, np.argmax(pred_val, axis = 1), average = "micro")
    print(f'F1={score}')
    return score

In [10]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

F1=0.7420705200450813


[I 2020-07-09 15:02:46,576] Finished trial#0 with value: 0.7420705200450813 with parameters: {'lambda_l1': 0.07915821496965882, 'lambda_l2': 1.0975514918158953e-08, 'num_leaves': 240, 'feature_fraction': 0.5035813049263763, 'bagging_fraction': 0.6178603117179465, 'bagging_freq': 5, 'min_child_samples': 34}. Best is trial#0 with value: 0.7420705200450813.


F1=0.745934632104331


[I 2020-07-09 15:04:37,252] Finished trial#1 with value: 0.745934632104331 with parameters: {'lambda_l1': 1.0841903571717806e-08, 'lambda_l2': 7.264300931817817e-06, 'num_leaves': 238, 'feature_fraction': 0.8876376963939318, 'bagging_fraction': 0.49943166784579035, 'bagging_freq': 2, 'min_child_samples': 80}. Best is trial#1 with value: 0.745934632104331.


F1=0.735952342617936


...

F1=0.7472226694574142


[I 2020-07-09 18:28:09,994] Finished trial#98 with value: 0.7472226694574142 with parameters: {'lambda_l1': 0.20282877522656026, 'lambda_l2': 7.253055309551843e-05, 'num_leaves': 141, 'feature_fraction': 0.7480813021060829, 'bagging_fraction': 0.838535192874358, 'bagging_freq': 5, 'min_child_samples': 52}. Best is trial#55 with value: 0.7588150056351635.


F1=0.7470616647882788


[I 2020-07-09 18:31:44,209] Finished trial#99 with value: 0.7470616647882788 with parameters: {'lambda_l1': 3.860921215338347e-08, 'lambda_l2': 1.289321054265063e-06, 'num_leaves': 158, 'feature_fraction': 0.8126035550903202, 'bagging_fraction': 0.9030543037234641, 'bagging_freq': 4, 'min_child_samples': 45}. Best is trial#55 with value: 0.7588150056351635.


Number of finished trials: 100
Best trial:
  Value: 0.7588150056351635
  Params: 
    lambda_l1: 0.05757233864212971
    lambda_l2: 4.644528267572721e-08
    num_leaves: 185
    feature_fraction: 0.8432452014952949
    bagging_fraction: 0.7795604743183974
    bagging_freq: 6
    min_child_samples: 61
